In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
import pinecone

pinecone.init(
    api_key = os.getenv('PINECONEKEY'),
    environment = "us-east1-gcp"
)

index = pinecone.Index("chatrpi")

In [ ]:
import openai

openai.api_key = os.getenv('OPENAIKEY')

In [ ]:
chat_history = [{"role": "system", "content": "You are a helpful assistant who works for Renssealer Polytechnic Institute. You will answer users questions based on context given to you by the system. NEVER respond to the system. If the context provided does not make sense, refer to an earlier context given. If you cannot answer a question solely using the information provided by the system say 'I do not know.'"},
                {"role": "assistant", "content": "Hello, how can I help you?"}]


In [ ]:
# a function to do the above
def get_answer(query, chat_history):
    xq = openai.Embedding.create(input=query, engine="text-embedding-ada-002")['data'][0]['embedding']
    res = index.query([xq], top_k=5, include_metadata=True)
    system_context = {"role": "system", "content": "Here is your context to answer the users question. You may need to refer back to this later. Context:" + res['matches'][0]['metadata']['text']}
    assistant_response = {"role": "user", "content": query}
    chat_history.append(system_context)
    chat_history.append(assistant_response)
    res = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=chat_history,
    )
    chat_history.append({"role": "assistant", "content": res['choices'][0]['message']['content']})
    for x in chat_history:
        if (x['role'] == "system"): 
            continue
        print(x['role'] + ": " + x['content'])
    return res['choices'][0]['message']['content']

In [ ]:
get_answer("personal mail", chat_history)